# Link Class Testing

Check if servers can be used to develop a reasonable network topology with reasonable bandwidth constraints.

### Import Link Class

In [108]:
import numpy as np
import itertools

class Link:
    """
    Link: used for representing the following regarding links between servers
        - Existence of link
        - Number of paths between two servers
        - Resource constraint of each link
        - How many resources have been reserved at each timestep
    """
    
    def __init__(self, servers, num_link, prob_link, lv_minmax):
        """
        Inputs:
        servers  - servers class holding server location information
        num_link - number of links each weak server should have with others
        prob_link - pdf of each possible count in num_link
        lv_minmax - uniform distribution range for link capacity (mbps) based on server level
        
        Attributes:
        valid_links = binary indication of which links exist in the system
        num_path = number of path available between any two servers
        lit_links = path indexed for 2 servers based on num_path
        
        """
        
        # Obtain distances between each server
        self.distances = self.calc_distance(servers)
        
        # invoke function for generating links
        self.valid_links = self.generate_links(servers, num_link, prob_link)
        
        # Generate resource capacities for each link
        self.rsrc_avail = self.link_resources(servers, lv_minmax)
        
        # invoke function for indicating which links are lit for each unique path
        self.path_graph = Graph(self.valid_links)
        
        # Count number of paths between two servers
        self.num_path = self.count_paths()
        
    
    """
    Initialization Functions
    """
        
    def calc_distance(self, servers):
        """
        Return S x S Matrix with distances between each server
        """
                
        # make list of locations
        locs = []
        x,y = np.zeros(len(servers)),np.zeros(len(servers))
        for i in range(len(servers)):
            x[i], y[i] = servers[i].locs[0], servers[i].locs[1]
            
        # Compute euclidean distance for every combination of servers        
        srv_dists = np.sqrt(np.square(x - x.reshape(-1,1)) + np.square(y - y.reshape(-1,1)))
        
        return srv_dists
        
    
    def generate_links(self, servers, num_link, prob_link):
        """
        Generate SxS np array for which links exist in the system
        For now every link in the system is bi-directional
        """
        
        valid_links = np.zeros((len(servers),len(servers)))

        # Obtain server level for each index
        svr_lvls = np.zeros(len(servers))
        for i in range(len(servers)):
            svr_lvls[i] = servers[i].level
        
        # Generate links between all level 3 and 2 servers bidirectional
        lvl23_idx = np.where(svr_lvls>1)
        mesh = np.array(np.meshgrid(lvl23_idx, lvl23_idx))
        combinations = mesh.T.reshape(-1, 2)
        valid_links[combinations[:,0],combinations[:,1]] = 1
        
        # Generate links between level 2 and closest level 1 servers
        # Generate links between level 1 servers to other level 1 servers
        lvl2_idx = np.where(svr_lvls==2)[0]
        lvl1_idx = np.where(svr_lvls==1)[0]
        
        for i in lvl1_idx:
            # From level 1 to level 2 servers
            dists = self.get_distance(s1_idx=i,s2_idx=None)
            mask = np.zeros(len(servers))
            mask[lvl2_idx] = 1
            dists_lv2 = dists * mask
            min_idx = np.where(dists_lv2==np.min(dists_lv2[np.nonzero(dists_lv2)]))[0][0]
            valid_links[i,min_idx], valid_links[min_idx,i] = 1,1
            
            # From level 1 to level 1 servers
            mask = np.zeros(len(servers))
            mask[lvl1_idx] = 1
            dists_lvl1 = dists * mask
            
            # Draw number of links to be made and iteratively generate
            lvl1_xn = np.random.choice(num_link, 1, p=prob_link)[0]

            for j in range(lvl1_xn):
                # If too many links already just break
                curr_lv1_links = np.sum(valid_links[i,lvl1_idx])
                lv1_serv_count = len(lvl1_idx)
                
                if curr_lv1_links >= lvl1_xn:
                    break
                elif np.sum(dists_lvl1) <= 0:
                    break
                
                # Draw link for minimum and remove lv1 server from contention
                min_idx = np.where(dists_lvl1==np.min(dists_lvl1[np.nonzero(dists_lvl1)]))[0][0]
                valid_links[i,min_idx], valid_links[min_idx,i] = 1,1
                dists_lvl1[min_idx] = 0
        
        # set identity to zero
        np.fill_diagonal(valid_links, 0)
        
        return valid_links
    
    def link_resources(self, servers, lv_minmax):
        """
        Sets each link resource capacity for each timestep to use as constraint
        
        Inputs:
        servers - list of server objects
        lv_minmax has 3 rows
        lv1_minmax - uniform resource range (mbps) for lv1 link
        lv2_minmax - uniform resource range (mbps) for lv2 link
        lv3_minmax - uniform resource range (mbps) for lv3 link
        """
        
        # check if link capacity has already been created for server pair
        checklist = np.zeros(self.valid_links.shape)
        
        rsrc_avail = np.copy(self.valid_links)
        
        for s1, s2 in itertools.product(range(self.valid_links.shape[0]),
                                        range(self.valid_links.shape[1])):
            if checklist[s1,s2] == 0 and self.valid_links[s1,s2] > 0:

                checklist[s1,s2], checklist[s2,s1] = 1,1

                # Make resource capacity based on level
                max_lvl = np.max([servers[s1].level, servers[s2].level]) - 1
                resource_draw = np.random.uniform(low = lv_minmax[max_lvl,0],
                                                      high = lv_minmax[max_lvl,1],
                                                      size = None)

                # List resource capacity, same bidirectional
                rsrc_avail[s1,s2] = resource_draw
                rsrc_avail[s2,s1] = resource_draw
                    
        return rsrc_avail
    
    def count_paths(self):
        """
        Given the graph object holding dictionary of possible paths, count number
        """
        
        path_counts = np.zeros(self.valid_links.shape)
        
        for s1, s2 in itertools.product(range(self.valid_links.shape[0]),
                                        range(self.valid_links.shape[1])):
            if s1 != s2:
                path_counts[s1,s2] = len(self.path_graph.path_dict[(s1,s2)])
                
        return path_counts
                
    
    """
    Utility Functions
    """
    
    def get_distance(self,s1_idx,s2_idx=None):
        """
        Returns distance between server s1 and server s2 (double)
        If only server s1 is inputted, returns distance to all other servers
        """
        
        if s2_idx is None:
            return self.distances[s1_idx]
        else:
            return self.distances[s1_idx][s2_idx]
    


### My edited version of pathfinder class

In [109]:
from collections import defaultdict 
import itertools

class Graph:
    """
    Given binary 2D transition matrix, give functionality for finding unique paths between 2 vertices
    Source: https://www.geeksforgeeks.org/find-paths-given-source-destination/
    
    Used for finding link paths between any two servers 
    """
    def __init__(self, adj_matrix):
        """
        Input - adj_matrix : adjacency matrix with links lit binary (1/0) np.ndarray
        """
        
        #No. of vertices 
        self.V= adj_matrix.shape[0]
        
        # default dictionary to store graph 
        self.graph = defaultdict(list) 
        
        # Add all of edges in adj_matrix to graph
        self.addEdges(adj_matrix)
        
        # Add dictionary to hold paths of each s1,s2 index
        self.path_dict = {}
        
        # Find unique paths and place in dictionary
        self.getAllPaths(adj_matrix)
    
    
    def addEdges(self,adj_matrix):
        """
        Take all edges of the adj_matrix and add them to graph attribute
        """
        for s1 in range(adj_matrix.shape[0]):
            for s2 in range(adj_matrix.shape[1]):
                if adj_matrix[s1,s2] > 0:
                    self.addEdge(s1,s2)
                
    
    def addEdge(self,u,v): 
        """
        # function to add an edge to graph 
        """
        self.graph[u].append(v) 
    
    
    def getPathsUtil(self, u, d, visited, path): 
  
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
  
        # If current vertex is same as destination, then print 
        # current path[] 
        if u == d: 
            # print(path)
            if path not in self.path_dict[self.curr_key]:
                self.path_dict[self.curr_key].append(path.copy())
        else: 
            # If current vertex is not destination 
            #Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]==False: 
                    self.getPathsUtil(i, d, visited, path) 
                      
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop() 
        visited[u]= False
        
   
    # Prints all paths from 's' to 'd' 
    def getPaths(self,s, d): 
    
        # Mark all the vertices as not visited 
        visited =[False]*(self.V) 
  
        # Create an array to store paths 
        path = [] 
  
        # Call the recursive helper function to print all paths 
        self.getPathsUtil(s, d,visited, path) 
        
    def getAllPaths(self, adj_matrix):
        """
        For every source, destination server combination, write in dictionary all possible paths
        """
        for s1,s2 in itertools.product(range(adj_matrix.shape[0]),range(adj_matrix.shape[1])):
            if s1 != s2:
                self.curr_key = (s1,s2)
                self.path_dict[self.curr_key] = []
                self.getPaths(s1,s2)

### Import Server Class and Generate Servers

In [114]:
from Server import Server

# Set params 
boundaries =np.array([[0,50],[0,50]])
num_resource = 2
weak_range = np.array([[2,8],[10,14]])
strong_range = np.array([[2,8],[10,14]])*2
timesteps = 3

# Number of each types of servers
num_server_l1 = 10
num_server_l2 = 3
num_server_l3 = 1

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []

for i in range(num_server_l1):
    servers_l1.append(Server(boundaries,level=1,rand_locs=True,locs=None))
    
for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    
for i in range(num_server_l3):
    servers_l3.append(Server(boundaries,level=3,rand_locs=True,locs=None))
    
# Append all servers together
servers = servers_l1 + servers_l2 + servers_l3

### Generate Links for given servers

In [115]:
# Set params
num_link = [0,1,2]
prob_link = [0.5,0.3,0.2]
lv_minmax = np.array(([[1,5],[5,10],[10,15]]))
timesteps = 3

links = Link(servers, num_link, prob_link, lv_minmax)
links.valid_links.shape

(14, 14)

In [116]:
links.path_graph.graph

defaultdict(list,
            {0: [10],
             1: [7, 12],
             2: [11],
             3: [4, 11],
             4: [3, 6, 11],
             5: [12],
             6: [4, 8, 11],
             7: [1, 12],
             8: [6, 9, 11],
             9: [8, 11],
             10: [0, 11, 12, 13],
             11: [2, 3, 4, 6, 8, 9, 10, 12, 13],
             12: [1, 5, 7, 10, 11, 13],
             13: [10, 11, 12]})

In [117]:
links.num_path

array([[ 0., 10.,  5., 25., 25.,  5., 25., 10., 25., 25.,  1.,  5.,  5.,
         5.],
       [10.,  0., 10., 50., 50.,  2., 50.,  2., 50., 50., 10., 10.,  2.,
        10.],
       [ 5., 10.,  0.,  5.,  5.,  5.,  5., 10.,  5.,  5.,  5.,  1.,  5.,
         5.],
       [25., 50.,  5.,  0.,  5., 25.,  8., 50., 10., 11., 25.,  5., 25.,
        25.],
       [25., 50.,  5.,  5.,  0., 25.,  7., 50.,  9., 10., 25.,  5., 25.,
        25.],
       [ 5.,  2.,  5., 25., 25.,  0., 25.,  2., 25., 25.,  5.,  5.,  1.,
         5.],
       [25., 50.,  5.,  8.,  7., 25.,  0., 50.,  7.,  8., 25.,  5., 25.,
        25.],
       [10.,  2., 10., 50., 50.,  2., 50.,  0., 50., 50., 10., 10.,  2.,
        10.],
       [25., 50.,  5., 10.,  9., 25.,  7., 50.,  0.,  5., 25.,  5., 25.,
        25.],
       [25., 50.,  5., 11., 10., 25.,  8., 50.,  5.,  0., 25.,  5., 25.,
        25.],
       [ 1., 10.,  5., 25., 25.,  5., 25., 10., 25., 25.,  0.,  5.,  5.,
         5.],
       [ 5., 10.,  1.,  5.,  5.,  5.,  5., 